In [1]:
'''
#UNCOMMENT AND RUN THESE TO INSTALL ANY MISSING PACKAGES
!apt-get install -q libgeos-3.5.0
!apt-get install -q libgeos-dev
!pip install -q https://github.com/matplotlib/basemap/archive/master.zip
!pip install -q pyproj==1.9.6
!pip install --no-binary shapely shapely --force
!apt-get install libproj-dev proj-data proj-bin  
!apt-get install libgeos-dev 
!pip install cython
!sudo pip install cartopy
!pip install geopandas
'''

'\n#UNCOMMENT AND RUN THESE TO INSTALL ANY MISSING PACKAGES\n!apt-get install -q libgeos-3.5.0\n!apt-get install -q libgeos-dev\n!pip install -q https://github.com/matplotlib/basemap/archive/master.zip\n!pip install -q pyproj==1.9.6\n!pip install --no-binary shapely shapely --force\n!apt-get install libproj-dev proj-data proj-bin  \n!apt-get install libgeos-dev \n!pip install cython\n!sudo pip install cartopy\n!pip install geopandas\n'

In [ ]:
from netCDF4 import Dataset as NetCDFFile # extract the nc4 file to get geography data
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import os, sys
import numpy
# sys.maxsize
numpy.set_printoptions(threshold=100)
 
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
 
#from google.colab import drive 
#drive.mount('/content/gdrive'); nb_path = '/content/notebooks'
#os.symlink('/content/gdrive/MyDrive/Colab Notebooks', nb_path); sys.path.insert(0, nb_path)  
 
 
#search for specific base file: find `conda info --base` -name epsg
os.environ["PROJ_LIB"] ='/opt/anaconda3/pkgs/proj4-5.2.0-h0a44026_1/share/proj/'
#from mpl_toolkits.basemap import Basemap #ploting the US map; install by `conda install -c anaconda basemap`
 
from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io import shapereader as shpreader
from cartopy.feature import ShapelyFeature

import plotly.graph_objects as go
import skimage.io as sio
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
try: 
    os.mkdir("data") 
except OSError as error: 
    print(error)  
try: 
    os.mkdir("globaldata") 
except OSError as error: 
    print(error) 

In [ ]:
'''
# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
 
fig = go.Figure(data=[go.Surface(z=z_data.values, colorscale = 'viridis')])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=600, height=500,
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene = dict(
                     xaxis = dict(nticks=6, range=[0,20],),
                     yaxis = dict(nticks=6, range=[0,20],),
                     zaxis = dict(nticks=15, range=[0,500],),)
)
 
fig.show()
'''

# 3D Plot with Contours

In [ ]:
import urllib

def download_data(date):
    geography_file_name = f'GRACEDADM_CLSM0125US_7D.A{date}.030.nc4'
    web_adress = f'https://nasagrace.unl.edu/data/{date}/'
    file = urllib.request.urlretrieve(web_adress + geography_file_name, f'data/{geography_file_name}')
download_data(20210809);

In [ ]:
geography_file = NetCDFFile("data/GRACEDADM_CLSM0125US_7D.A20210809.030.nc4")
k = 1
# unpack data into variables
lat = geography_file['lat'][::k]
lon = geography_file['lon'][::k]

gws = geography_file['gws_inst'][0, ::k, ::k]
rtzsm = geography_file['rtzsm_inst'][0, ::k, ::k]
sfsm = geography_file['sfsm_inst'][0, ::k, ::k]

# create a meshgrid for plotting
Lat, Lon = np.meshgrid(lat, lon)

# put into dataframe. plotly likes dataframes
data = pd.DataFrame(np.c_[Lat.T.flatten(), Lon.T.flatten(), gws.flatten()], columns=['Lat', 'Lon', 'gws'])

# here we are using the meshgrids (notice "Lon", not "lon")
# in order to display different levels we simply subtract a fixed value from all z coordinates of a layer
# since we do not use special map projections here (or, to be me more precise, we use the equidistant cyllindrical projection),
# we have a situation where the height of surfaces peak are too big compared to the scale of map
# so we scale down the height manually
gws_scale, rtzsm_scale, sfsm_scale = 1, 1, 1
rtzsm_shift, gws_shift = 60, 60

fig = go.Figure(data=[go.Surface(x=Lon, y=Lat, z=gws.filled(fill_value=np.nan).T / gws_scale,
                                 name='Ground water levels', colorscale='RdBu', showscale=False)])

fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))


 

fig.update_layout(title='USA map of ground waters', autosize=False,
                  width=500, height=500,
                  margin=dict(l=0, r=0, b=0, t=50), 
                  scene=dict(
                      #aspectmode='data',
                             xaxis=dict(title=dict(text='Lon'), 
                                        autorange='reversed'
                                       ),
                             yaxis=dict(title=dict(text='Lat'), 
                                        autorange='reversed'
                                       ),
                             zaxis=dict(title=dict(text=''), nticks=20, range=[-80, 160]),
                            ),
)
fig.show()


# Static 3D Plot with All 3 Layers Stacked

In [ ]:
# first we read the state boundaries data
from cartopy.io import shapereader
from cartopy.mpl.patch import geos_to_path
import geopandas

# get natural earth data (http://www.naturalearthdata.com/)

# get borders
resolution = '110m'
category = 'cultural'
name = 'admin_1_states_provinces'

shpfilename = shapereader.natural_earth(resolution, category, name)

# read the shapefile using geopandas
df = geopandas.read_file(shpfilename)

# remove Hawaii and Alaska
df = df[~((df['name'] == 'Alaska') | (df['name'] == 'Hawaii'))]

In [ ]:
geography_file = NetCDFFile("data/GRACEDADM_CLSM0125US_7D.A20210809.030.nc4")
k = 1
# unpack data into variables
lat = geography_file['lat'][::k]
lon = geography_file['lon'][::k]
gws = geography_file['gws_inst'][0, ::k, ::k]
rtzsm = geography_file['rtzsm_inst'][0, ::k, ::k]
sfsm = geography_file['sfsm_inst'][0, ::k, ::k]

# create a meshgrid for plotting
Lat, Lon = np.meshgrid(lat, lon)

In [ ]:
# here we are using the meshgrids (notice "Lon", not "lon")
# in order to display different levels we simply subtract a fixed value from all z coordinates of a layer
# since we do not use special map projections here (or, to be me more precise, we use the equidistant cyllindrical projection),
# we have a situation where the height of surfaces peak are too big compared to the scale of map
# so we scale down the height manually
gws_scale, rtzsm_scale, sfsm_scale = 20, 20, 20
rtzsm_shift, gws_shift = 20, 40

fig = go.Figure(data=[go.Surface(x=Lon, y=Lat, z=gws.filled(fill_value=np.nan).T / gws_scale - gws_shift,
                                 name='Ground water levels', colorscale='RdBu', showscale=False),
                      go.Surface(x=Lon, y=Lat, z=rtzsm.T.filled(fill_value=np.nan) / rtzsm_scale - rtzsm_shift,
                                 name='Root zone soil moisture', colorscale='PuOr', showscale=False),
                      go.Surface(x=Lon, y=Lat, z=sfsm.T.filled(fill_value=np.nan) / sfsm_scale,
                                 name='Surface soil moisture', colorscale='PiYG', showscale=False)])

# now we add all states to the plot one by one
for poly in df['geometry']:
    # get the geometry object, convert it to path object
    path = geos_to_path(poly)[0]
    # now obtain x and y coordinates from the path object
    xs, ys = path.to_polygons()[0].T
    # this is how how high the borders will be shown
    # we will display them slighly higher than the corresponding map layer
    # for better visibility
    zs = np.zeros_like(xs) + 2 - gws_shift
    fig.add_scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode='lines',
        name='',
        showlegend=False,
        line=dict(width=3, color='black')
    )
    
    # same, but for root zone moisture
    path = geos_to_path(poly)[0]
    xs, ys = path.to_polygons()[0].T
    zs = np.zeros_like(xs) + 2 - rtzsm_shift
    fig.add_scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode='lines',
        name='',
        showlegend=False,
        line=dict(width=3, color='black')
    )
    
    # same, but for surface soil moisture
    path = geos_to_path(poly)[0]
    xs, ys = path.to_polygons()[0].T
    zs = np.zeros_like(xs) + 2
    fig.add_scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode='lines',
        name='',
        showlegend=False,
        line=dict(width=3, color='black')
    )
fig.update_layout(title='USA map of ground waters', autosize=False,
                  width=600, height=500,
                  margin=dict(l=0, r=0, b=0, t=50), 
                  scene=dict(
                      #aspectmode='data',
                             xaxis=dict(title=dict(text='Lon'), 
                                        autorange='reversed'
                                       ),
                             yaxis=dict(title=dict(text='Lat'), 
                                        autorange='reversed'
                                       ),
                             zaxis=dict(title=dict(text='')),
                            ),
)
fig.show()

# Animated 3D Plot with All 3 Layers Stacked

In [ ]:
gws_scale, rtzsm_scale, sfsm_scale = 20, 20, 20
rtzsm_shift, gws_shift = 20, 40

frames_list = []
# this snippet with parameters was taken from the official documentation
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Date: ",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

#downloading three more datasets

download_data(20210823)
download_data(20210712)
download_data(20210628)

date = 20210823
geography_file_name = f'GRACEDADM_CLSM0125US_7D.A{date}.030.nc4'
web_adress = f'https://nasagrace.unl.edu/data/{date}/'

file = urllib.request.urlretrieve(web_adress + geography_file_name, f'data/{geography_file_name}')

for geography_file_name in os.listdir('data'):
    geography_file = NetCDFFile(f'data/{geography_file_name}')
    date = geography_file_name[25:33]
    
    k = 1
    # unpack data into variables
    lat = geography_file['lat'][::k]
    lon = geography_file['lon'][::k]
    
    gws = geography_file['gws_inst'][0, ::k, ::k]
    rtzsm = geography_file['rtzsm_inst'][0, ::k, ::k]
    sfsm = geography_file['sfsm_inst'][0, ::k, ::k]

    # create a meshgrid for plotting
    Lat, Lon = np.meshgrid(lat, lon)

    frames_list.append(go.Frame(data=[go.Surface(x=Lon, y=Lat, z=gws.filled(np.nan).T / gws_scale - gws_shift,
                                                 name='Ground water levels', colorscale='RdBu', showscale=False),
                                      go.Surface(x=Lon, y=Lat, z=rtzsm.T.filled(np.nan) / rtzsm_scale - rtzsm_shift,
                                                 name='Root zone soil moisture', colorscale='PuOr', showscale=False),
                                      go.Surface(x=Lon, y=Lat, z=sfsm.T.filled(np.nan) / sfsm_scale,
                                                 name='Surface soil moisture', colorscale='PiYG', showscale=False)],
                                name=date
                               )
                      )
    
    slider_step = {"args": [
        [date],
        {"frame": {"duration": 300, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": date,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)

fig = go.Figure(frames=frames_list, layout={'sliders': [sliders_dict]})
    
# now we add all states to the plot one by one
for poly in df['geometry']:
    # increase to reduce details and increase speed
    geo_simple_threshold = 0.1
    # get the geometry object, convert it to path object
    path = geos_to_path(poly.simplify(geo_simple_threshold))[0]
    # now obtain x and y coordinates from the path object
    xs, ys = path.to_polygons()[0].T
    # this is how how high the borders will be shown
    # we will display them slighly higher than the corresponding map layer
    # for better visibility
    zs = np.zeros_like(xs) + 2 - gws_shift
    fig.add_scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode='lines',
        name='',
        showlegend=False,
        line=dict(width=5, color='black')
    )
    
    # same, but for root zone moisture
    path = geos_to_path(poly.simplify(geo_simple_threshold))[0]
    xs, ys = path.to_polygons()[0].T
    zs = np.zeros_like(xs) + 2 - rtzsm_shift
    fig.add_scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode='lines',
        name='',
        showlegend=False,
        line=dict(width=5, color='black')
    )
    
    # same, but for surface soil moisture
    path = geos_to_path(poly.simplify(geo_simple_threshold))[0]
    xs, ys = path.to_polygons()[0].T
    zs = np.zeros_like(xs) + 2
    fig.add_scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode='lines',
        name='',
        showlegend=False,
        line=dict(width=3, color='black')
    )

fig.update_layout(title='USA map of ground waters', autosize=False,
                  width=600, height=600,
                  margin=dict(l=0, r=0, b=0, t=50), 
                  scene=dict(aspectmode='data',
                             xaxis=dict(title=dict(text='Lon'), autorange='reversed'),
                             yaxis=dict(title=dict(text='Lat'), autorange='reversed'),
                             zaxis=dict(title=dict(text='')),
                            ),
                  # this snippet with parameters was taken from the official documentation
                  updatemenus=[{'buttons': [
                                {
                                    "args": [None, {"frame": {"duration": 500, "redraw": True},
                                                    "fromcurrent": True, "transition": {"duration": 300,
                                                                                        "easing": "quadratic-in-out"}}],
                                    "label": "Play",
                                    "method": "animate"
                                },
                                {
                                    "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}],
                                    "label": "Pause",
                                    "method": "animate"
                                }
                            ],
                                'direction': 'left','pad': {'r': 10, 't': 87},'showactive': True,
                                'type': 'buttons','x': 0.1,'xanchor': 'right','y': 0.2,'yanchor': 'top'}],
)


# Layer by Layer Static Position Animation

In [ ]:
import os

gws_scale, rtzsm_scale, sfsm_scale = 20, 20, 20
rtzsm_shift, gws_shift = 20, 40

frames_list = []
# this snippet with parameters was taken from the official documentation
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Currently: ",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

for geography_file_name in os.listdir('data'):
    geography_file = NetCDFFile(f'data/{geography_file_name}')
    date = geography_file_name[25:33]
    date = date[:4] + '-' + date[4:6] + '-' + date[6:]
    
    k = 1
    # unpack data into variables
    lat = geography_file['lat'][::k]
    lon = geography_file['lon'][::k]
    
    gws = geography_file['gws_inst'][0, ::k, ::k]
    rtzsm = geography_file['rtzsm_inst'][0, ::k, ::k]
    sfsm = geography_file['sfsm_inst'][0, ::k, ::k]

    # create a meshgrid for plotting
    Lat, Lon = np.meshgrid(lat, lon)

    frames_list.append(go.Frame(data=[go.Surface(x=Lon, y=Lat, z=gws.filled(np.nan).T / gws_scale,
                                                 name='Ground water levels', colorscale='RdBu', showscale=False)],
                                name=f'gws {date}'))
    slider_step = {"args": [[f'gws {date}'], {"frame": {"duration": 300, "redraw": True}, "mode": "immediate", "transition": {"duration": 300}}],
        "label": f'gws {date}', "method": "animate"}
    sliders_dict["steps"].append(slider_step)
    
    frames_list.append(go.Frame(data=[go.Surface(x=Lon, y=Lat, z=rtzsm.T.filled(np.nan) / rtzsm_scale,
                                                 name='Root zone soil moisture', colorscale='PuOr', showscale=False)],
                                name=f'rtzsm {date}'))
    slider_step = {"args": [[f'rtzsm {date}'], {"frame": {"duration": 300, "redraw": True}, "mode": "immediate", "transition": {"duration": 300}}],
        "label": f'rtzsm {date}', "method": "animate"}
    sliders_dict["steps"].append(slider_step)
    
    frames_list.append(go.Frame(data=[go.Surface(x=Lon, y=Lat, z=sfsm.T.filled(np.nan) / sfsm_scale,
                                                 name='Surface soil moisture', colorscale='PiYG', showscale=False)],
                                name=f'sfsm {date}'))
    slider_step = {"args": [[f'sfsm {date}'], {"frame": {"duration": 300, "redraw": True}, "mode": "immediate", "transition": {"duration": 300}}],
        "label": f'sfsm {date}', "method": "animate"}
    sliders_dict["steps"].append(slider_step)

fig = go.Figure(frames=frames_list, layout={'sliders': [sliders_dict]})
    
# now we add all states to the plot one by one
for poly in df['geometry']:
    # increase to reduce details and increase speed
    geo_simple_threshold = 0.1
    # get the geometry object, convert it to path object
    path = geos_to_path(poly.simplify(geo_simple_threshold))[0]
    # now obtain x and y coordinates from the path object
    xs, ys = path.to_polygons()[0].T
    # this is how how high the borders will be shown
    # we will display them slighly higher than the corresponding map layer
    # for better visibility
    zs = np.zeros_like(xs) + 2
    fig.add_scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode='lines',
        name='',
        showlegend=False,
        line=dict(width=5, color='black')
    )

fig.update_layout(title='USA map of ground waters', autosize=False,
                  width=500, height=500,
                  margin=dict(l=0, r=0, b=0, t=50), 
                  scene=dict(aspectmode='data',
                             xaxis=dict(title=dict(text='Lon'), autorange='reversed'),
                             yaxis=dict(title=dict(text='Lat'), autorange='reversed'),
                             zaxis=dict(title=dict(text=''), range=[-20, 20]),
                            ),
                  # this snippet with parameters was taken from the official documentation
                  updatemenus=[{'buttons': [
                                {
                                    "args": [None, {"frame": {"duration": 500, "redraw": True},
                                                    "fromcurrent": True, "transition": {"duration": 300,
                                                                                        "easing": "quadratic-in-out"}}],
                                    "label": "Play",
                                    "method": "animate"
                                },
                                {
                                    "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}],
                                    "label": "Pause",
                                    "method": "animate"
                                }
                            ],
                                'direction': 'left','pad': {'r': 10, 't': 87},'showactive': True,
                                'type': 'buttons','x': 0.1,'xanchor': 'right','y': 0.2,'yanchor': 'top'}],
)

fig.show()

# Layer by Layer Dynamic Position Animation

In [ ]:
from shapely.ops import cascaded_union

# create a single polygon for the US borders
outline = cascaded_union(df['geometry'])

xs, ys = geos_to_path(outline)[0].to_polygons()[0].T
zs = np.zeros_like(xs) + 2

In [ ]:
import os

gws_scale, rtzsm_scale, sfsm_scale = 20, 20, 20
rtzsm_shift, gws_shift = 20, 40

frames_list = []
# this snippet with parameters was taken from the official documentation
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Currently: ",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# baseframe = go.Frame(data=[go.Scatter3d(x=xs, y=ys, z=zs, mode='lines', name='', showlegend=False,
#                                                   line=dict(width=5, color='black'))])

for geography_file_name in os.listdir('data'):
    geography_file = NetCDFFile(f'data/{geography_file_name}')
    date = geography_file_name[25:33]
    date = date[:4] + '-' + date[4:6] + '-' + date[6:]
    
    k = 1
    # unpack data into variables
    lat = geography_file['lat'][::k]
    lon = geography_file['lon'][::k]
    
    gws = geography_file['gws_inst'][0, ::k, ::k]
    rtzsm = geography_file['rtzsm_inst'][0, ::k, ::k]
    sfsm = geography_file['sfsm_inst'][0, ::k, ::k]

    # create a meshgrid for plotting
    Lat, Lon = np.meshgrid(lat, lon)

    frames_list.append(go.Frame(data=[go.Surface(x=Lon, y=Lat, z=gws.filled(np.nan).T / gws_scale - gws_shift,
                                                 name='data', colorscale='RdBu', showscale=False),
                                      go.Scatter3d(x=xs, y=ys, z=zs - gws_shift, mode='lines', name='outline', showlegend=False,
                                                  line=dict(width=4, color='black'))],
                                traces=[0, 1],
                                group='gws',
                                name=f'gws {date}'))
    slider_step = {"args": [[f'gws {date}'], {"frame": {"duration": 300, "redraw": True}, "mode": "immediate", "transition": {"duration": 300}}],
        "label": f'gws {date}', "method": "animate"}
    
    sliders_dict["steps"].append(slider_step)
    
    frames_list.append(go.Frame(data=[go.Surface(x=Lon, y=Lat, z=rtzsm.T.filled(np.nan) / rtzsm_scale - rtzsm_shift,
                                                 name='data', colorscale='PuOr', showscale=False),
                                     go.Scatter3d(x=xs, y=ys, z=zs - rtzsm_shift, mode='lines', showlegend=False,
                                                  line=dict(width=4, color='black'))],
                                traces=[0, 1],
                                group='rtzsm',
                                name=f'rtzsm {date}'))
    slider_step = {"args": [[f'rtzsm {date}'], {"frame": {"duration": 300, "redraw": True}, "mode": "immediate", "transition": {"duration": 300}}],
        "label": f'rtzsm {date}', "method": "animate"}
    sliders_dict["steps"].append(slider_step)
    
    frames_list.append(go.Frame(data=[go.Surface(x=Lon, y=Lat, z=sfsm.T.filled(np.nan) / sfsm_scale,
                                                 name='data', colorscale='PiYG', showscale=False),
                                     go.Scatter3d(x=xs, y=ys, z=zs, mode='lines', showlegend=False,
                                                  line=dict(width=4, color='black'))],
                                traces=[0, 1],
                                group='sfsm',
                                name=f'sfsm {date}'))
    slider_step = {"args": [[f'sfsm {date}'], {"mode": "immediate", }],
        "label": f'sfsm {date}', "method": "animate"}
    sliders_dict["steps"].append(slider_step)

fig = go.Figure(
    data=[go.Surface(x=[], y=[], z=[],
                     name='data', colorscale='Greens', showscale=False),
          go.Scatter3d(x=xs, y=ys, z=zs, mode='lines', name='outline', showlegend=False,
                       line=dict(width=4, color='black'))],
    frames=frames_list,
    layout=dict(sliders=[sliders_dict])
)
# fig.add_scatter3d(x=[-100], y=[30], z=[0], mode='markers', marker=dict(size=0),)

fig.update_layout(title='USA map of ground waters', autosize=False,
                  width=800, height=500,
                  margin=dict(l=0, r=0, b=0, t=50), 
                  scene=dict(aspectmode='manual', aspectratio=dict(x=1, y=0.5, z=0.5),
                             xaxis=dict(title=dict(text='Lon'), autorange='reversed'),
                             yaxis=dict(title=dict(text='Lat'), autorange='reversed'),
                             zaxis=dict(title=dict(text=''), range=[-gws_shift-5, 15]),
                            ),
                  # this snippet with parameters was taken from the official documentation
                  updatemenus=[{'buttons': [
                                {
                                    "args": [None, {"frame": {"duration": 500, "redraw": True},
                                                    "fromcurrent": True,
                                                    "transition": {"duration": 300, "easing": "quadratic-in-out"}}],
                                    "label": "Play",
                                    "method": "animate"
                                },
                                {
                                    "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}],
                                    "label": "Pause",
                                    "method": "animate"
                                }
                            ],
                                'direction': 'left','pad': {'r': 10, 't': 87},'showactive': True,
                                'type': 'buttons','x': 0.1,'xanchor': 'right','y': 0.2,'yanchor': 'top'}],
)

fig.show()